In [1]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Class').getOrCreate()
spark


In [4]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import IndexToString
from pyspark.ml.feature import MinMaxScaler

In [142]:
path = "Udemy_datas/"
df = spark.read.csv(path + "Toddler Autism dataset July 2018.csv",inferSchema=True,header=True)

In [143]:
df.limit(5).toPandas()

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Qchat-10-Score,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Who completed the test,Class/ASD Traits
0,1,0,0,0,0,0,0,1,1,0,1,28,3,f,middle eastern,yes,no,family member,No
1,2,1,1,0,0,0,1,1,0,0,0,36,4,m,White European,yes,no,family member,Yes
2,3,1,0,0,0,0,0,1,1,0,1,36,4,m,middle eastern,yes,no,family member,Yes
3,4,1,1,1,1,1,1,1,1,1,1,24,10,m,Hispanic,no,no,family member,Yes
4,5,1,1,0,1,1,1,1,1,1,1,20,9,f,White European,no,yes,family member,Yes


In [144]:
df.printSchema()

root
 |-- Case_No: integer (nullable = true)
 |-- A1: integer (nullable = true)
 |-- A2: integer (nullable = true)
 |-- A3: integer (nullable = true)
 |-- A4: integer (nullable = true)
 |-- A5: integer (nullable = true)
 |-- A6: integer (nullable = true)
 |-- A7: integer (nullable = true)
 |-- A8: integer (nullable = true)
 |-- A9: integer (nullable = true)
 |-- A10: integer (nullable = true)
 |-- Age_Mons: integer (nullable = true)
 |-- Qchat-10-Score: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Ethnicity: string (nullable = true)
 |-- Jaundice: string (nullable = true)
 |-- Family_mem_with_ASD: string (nullable = true)
 |-- Who completed the test: string (nullable = true)
 |-- Class/ASD Traits : string (nullable = true)



In [145]:
dependent_var = "Class/ASD Traits "

In [146]:
renamed = df.withColumn("label_str",df[dependent_var].cast(StringType()))


In [147]:
renamed.limit(5).toPandas()

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Qchat-10-Score,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Who completed the test,Class/ASD Traits,label_str
0,1,0,0,0,0,0,0,1,1,0,1,28,3,f,middle eastern,yes,no,family member,No,No
1,2,1,1,0,0,0,1,1,0,0,0,36,4,m,White European,yes,no,family member,Yes,Yes
2,3,1,0,0,0,0,0,1,1,0,1,36,4,m,middle eastern,yes,no,family member,Yes,Yes
3,4,1,1,1,1,1,1,1,1,1,1,24,10,m,Hispanic,no,no,family member,Yes,Yes
4,5,1,1,0,1,1,1,1,1,1,1,20,9,f,White European,no,yes,family member,Yes,Yes


In [148]:
indexer  = StringIndexer(inputCol="label_str",outputCol="label")
indexed  = indexer.fit(renamed).transform(renamed)

In [149]:
indexed.limit(5).toPandas()

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,Age_Mons,Qchat-10-Score,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Who completed the test,Class/ASD Traits,label_str,label
0,1,0,0,0,0,0,0,1,1,0,...,28,3,f,middle eastern,yes,no,family member,No,No,1.0
1,2,1,1,0,0,0,1,1,0,0,...,36,4,m,White European,yes,no,family member,Yes,Yes,0.0
2,3,1,0,0,0,0,0,1,1,0,...,36,4,m,middle eastern,yes,no,family member,Yes,Yes,0.0
3,4,1,1,1,1,1,1,1,1,1,...,24,10,m,Hispanic,no,no,family member,Yes,Yes,0.0
4,5,1,1,0,1,1,1,1,1,1,...,20,9,f,White European,no,yes,family member,Yes,Yes,0.0


In [150]:
df.schema['Case_No'].dataType

IntegerType

In [151]:
input_columns = df.columns[1:-1]

In [152]:
input_columns

['A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'A10',
 'Age_Mons',
 'Qchat-10-Score',
 'Sex',
 'Ethnicity',
 'Jaundice',
 'Family_mem_with_ASD',
 'Who completed the test']

In [153]:
string_inputs = []
num_inputs = []
for column in input_columns:
    if str(df.schema[column].dataType)=='StringType':
        indexer = StringIndexer(inputCol= column,outputCol=column + "_num")
        indexed = indexer.fit(indexed).transform(indexed)
        new_column_name = column + "_num"
        string_inputs.append(new_column_name)
    else:
        num_inputs.append(column)

In [154]:
string_inputs

['Sex_num',
 'Ethnicity_num',
 'Jaundice_num',
 'Family_mem_with_ASD_num',
 'Who completed the test_num']

In [155]:
num_inputs

['A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'A10',
 'Age_Mons',
 'Qchat-10-Score']

In [156]:
indexed.printSchema()

root
 |-- Case_No: integer (nullable = true)
 |-- A1: integer (nullable = true)
 |-- A2: integer (nullable = true)
 |-- A3: integer (nullable = true)
 |-- A4: integer (nullable = true)
 |-- A5: integer (nullable = true)
 |-- A6: integer (nullable = true)
 |-- A7: integer (nullable = true)
 |-- A8: integer (nullable = true)
 |-- A9: integer (nullable = true)
 |-- A10: integer (nullable = true)
 |-- Age_Mons: integer (nullable = true)
 |-- Qchat-10-Score: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Ethnicity: string (nullable = true)
 |-- Jaundice: string (nullable = true)
 |-- Family_mem_with_ASD: string (nullable = true)
 |-- Who completed the test: string (nullable = true)
 |-- Class/ASD Traits : string (nullable = true)
 |-- label_str: string (nullable = true)
 |-- label: double (nullable = false)
 |-- Sex_num: double (nullable = false)
 |-- Ethnicity_num: double (nullable = false)
 |-- Jaundice_num: double (nullable = false)
 |-- Family_mem_with_ASD_num: double

In [157]:
indexed.limit(5).toPandas()

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,Family_mem_with_ASD,Who completed the test,Class/ASD Traits,label_str,label,Sex_num,Ethnicity_num,Jaundice_num,Family_mem_with_ASD_num,Who completed the test_num
0,1,0,0,0,0,0,0,1,1,0,...,no,family member,No,No,1.0,1.0,2.0,1.0,0.0,0.0
1,2,1,1,0,0,0,1,1,0,0,...,no,family member,Yes,Yes,0.0,0.0,0.0,1.0,0.0,0.0
2,3,1,0,0,0,0,0,1,1,0,...,no,family member,Yes,Yes,0.0,0.0,2.0,1.0,0.0,0.0
3,4,1,1,1,1,1,1,1,1,1,...,no,family member,Yes,Yes,0.0,0.0,5.0,0.0,0.0,0.0
4,5,1,1,0,1,1,1,1,1,1,...,yes,family member,Yes,Yes,0.0,1.0,0.0,0.0,1.0,0.0


In [158]:
indexed.printSchema()

root
 |-- Case_No: integer (nullable = true)
 |-- A1: integer (nullable = true)
 |-- A2: integer (nullable = true)
 |-- A3: integer (nullable = true)
 |-- A4: integer (nullable = true)
 |-- A5: integer (nullable = true)
 |-- A6: integer (nullable = true)
 |-- A7: integer (nullable = true)
 |-- A8: integer (nullable = true)
 |-- A9: integer (nullable = true)
 |-- A10: integer (nullable = true)
 |-- Age_Mons: integer (nullable = true)
 |-- Qchat-10-Score: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Ethnicity: string (nullable = true)
 |-- Jaundice: string (nullable = true)
 |-- Family_mem_with_ASD: string (nullable = true)
 |-- Who completed the test: string (nullable = true)
 |-- Class/ASD Traits : string (nullable = true)
 |-- label_str: string (nullable = true)
 |-- label: double (nullable = false)
 |-- Sex_num: double (nullable = false)
 |-- Ethnicity_num: double (nullable = false)
 |-- Jaundice_num: double (nullable = false)
 |-- Family_mem_with_ASD_num: double

In [159]:
skew = indexed.agg(skewness(indexed['Age_Mons'])).collect()

In [160]:
t = {}

In [161]:
t['Age_Mons'] = indexed.approxQuantile('Age_Mons',[0.001,0.99],0.25)

In [162]:
t['Age_Mons']

[12.0, 36.0]

In [163]:
d = {}
for col in num_inputs:
    d[col] = indexed.approxQuantile(col,[0.001,0.99],0.25)

for col in num_inputs:
    skew = indexed.agg(skewness(col)).collect()
    skew = skew[0][0]
    if skew > 1:
        indexed = indexed.withColumn(col,log(when(indexed[col]<d[col][0],d[col][0])\
                 .when(indexed[col]>d[col][1],d[col][1]).otherwise(indexed[col]+1)).alias(col))
        print( col,"has been treated : positive skew : ",skew)
    elif skew < -1:
        indexed = indexed.withColumn(col,exp(when(indexed[col]<d[col][0],d[col][0])\
                 .when(indexed[col]>d[col][1],d[col][1]).otherwise(indexed[col]+1)).alias(col))
        print( col,"has been treated : neg skew : ",skew)                   
    

In [174]:
df.select([min(c).alias(c) for c in df.columns if c in num_inputs ]).show()


+---+---+---+---+---+---+---+---+---+---+--------+--------------+
| A1| A2| A3| A4| A5| A6| A7| A8| A9|A10|Age_Mons|Qchat-10-Score|
+---+---+---+---+---+---+---+---+---+---+--------+--------------+
|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|      12|             0|
+---+---+---+---+---+---+---+---+---+---+--------+--------------+



In [182]:
minimums =  df.select([min(c).alias(c) for c in df.columns if c in num_inputs ])


In [183]:
min_array = minimums.select(array(num_inputs).alias('mins'))

In [187]:
min_array.toPandas()

,mins
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0]"


In [188]:
df_minimum = min_array.select(array_min(min_array.mins)).collect()

In [189]:
df_minimum[0][0]

0

In [190]:
features_list = num_inputs + string_inputs
assembler = VectorAssembler(inputCols=features_list,outputCol="features")
output = assembler.transform(indexed).select('features','label')

In [191]:
output.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(17,[6,7,9,10,11,...|  1.0|
|(17,[0,1,5,6,10,1...|  0.0|
|(17,[0,6,7,9,10,1...|  0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|
|[1.0,1.0,0.0,1.0,...|  0.0|
|[1.0,1.0,0.0,0.0,...|  0.0|
|(17,[0,3,4,5,8,10...|  0.0|
|(17,[1,4,6,7,8,9,...|  0.0|
|(17,[6,9,10,11,13...|  1.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[1.0,0.0,0.0,1.0,...|  0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|
|(17,[10,12,13,14]...|  1.0|
|[1.0,1.0,1.0,1.0,...|  0.0|
|(17,[10,13],[18.0...|  1.0|
|(17,[0,1,2,4,6,7,...|  0.0|
|(17,[10,13,15],[3...|  1.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|(17,[0,4,9,10,11,...|  1.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [192]:
scaler = MinMaxScaler(inputCol='features',outputCol='scale_features',min = 0,max = 1000)
scaler_model = scaler.fit(output)
scaled_data = scaler_model.transform(output)

In [193]:
scaled_data.show()

+--------------------+-----+--------------------+
|            features|label|      scale_features|
+--------------------+-----+--------------------+
|(17,[6,7,9,10,11,...|  1.0|[0.0,0.0,0.0,0.0,...|
|(17,[0,1,5,6,10,1...|  0.0|[1000.0,1000.0,0....|
|(17,[0,6,7,9,10,1...|  0.0|[1000.0,0.0,0.0,0...|
|[1.0,1.0,1.0,1.0,...|  0.0|[1000.0,1000.0,10...|
|[1.0,1.0,0.0,1.0,...|  0.0|[1000.0,1000.0,0....|
|[1.0,1.0,0.0,0.0,...|  0.0|[1000.0,1000.0,0....|
|(17,[0,3,4,5,8,10...|  0.0|[1000.0,0.0,0.0,1...|
|(17,[1,4,6,7,8,9,...|  0.0|[0.0,1000.0,0.0,0...|
|(17,[6,9,10,11,13...|  1.0|[0.0,0.0,0.0,0.0,...|
|[1.0,1.0,1.0,0.0,...|  0.0|[1000.0,1000.0,10...|
|[1.0,0.0,0.0,1.0,...|  0.0|[1000.0,0.0,0.0,1...|
|[1.0,1.0,1.0,1.0,...|  0.0|[1000.0,1000.0,10...|
|(17,[10,12,13,14]...|  1.0|[0.0,0.0,0.0,0.0,...|
|[1.0,1.0,1.0,1.0,...|  0.0|[1000.0,1000.0,10...|
|(17,[10,13],[18.0...|  1.0|[0.0,0.0,0.0,0.0,...|
|(17,[0,1,2,4,6,7,...|  0.0|[1000.0,1000.0,10...|
|(17,[10,13,15],[3...|  1.0|[0.0,0.0,0.0,0.0,...|


In [194]:
final_data = scaled_data.select('label','scale_features')
final_data = final_data.withColumnRenamed("scale_features",'features')

In [195]:
final_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|[0.0,0.0,0.0,0.0,...|
|  0.0|[1000.0,1000.0,0....|
|  0.0|[1000.0,0.0,0.0,0...|
|  0.0|[1000.0,1000.0,10...|
|  0.0|[1000.0,1000.0,0....|
|  0.0|[1000.0,1000.0,0....|
|  0.0|[1000.0,0.0,0.0,1...|
|  0.0|[0.0,1000.0,0.0,0...|
|  1.0|[0.0,0.0,0.0,0.0,...|
|  0.0|[1000.0,1000.0,10...|
|  0.0|[1000.0,0.0,0.0,1...|
|  0.0|[1000.0,1000.0,10...|
|  1.0|[0.0,0.0,0.0,0.0,...|
|  0.0|[1000.0,1000.0,10...|
|  1.0|[0.0,0.0,0.0,0.0,...|
|  0.0|[1000.0,1000.0,10...|
|  1.0|[0.0,0.0,0.0,0.0,...|
|  0.0|[1000.0,1000.0,10...|
|  1.0|[1000.0,0.0,0.0,0...|
|  0.0|[1000.0,1000.0,10...|
+-----+--------------------+
only showing top 20 rows



In [203]:
train,test = final_data.randomSplit([0.7,0.3])

In [204]:
train.count()

749

In [205]:
test.count()

305

# Logistic Regression

In [206]:
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [207]:
Bin_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction')
Mc_evaluator = MulticlassClassificationEvaluator(metricName='accuracy')

In [213]:
#initialize and fir training model
classifier = LogisticRegression()
fitModel = classifier.fit(train)
try:
    #Evaluation for binary classification 
    predictionAndLabels = fitModel.evaluate(test)
    #predictionAndLabels = predictionAndLabels.predictions.select("label",'prediction')
    auc = Bin_evaluator.evaluate(predictionAndLabels)
    print("AUC: ",auc)
    raise  print("error")
# Evvaluation for Multiclass
predictions = fitModel.transform(test)
accuracy = (Mc_evaluator.evaluate(predictions))*100
print('Accuracy is {0:.2f}'.format(accuracy),'%')

SyntaxError: invalid syntax (<ipython-input-213-9c80e681d7d4>, line 12)